# Artifical Neural Network with TensorFlow

In [4]:
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
import pandas
from tensorflow import keras

In [5]:
def generate_one_hot(value):
    one_hot = [0.0]*26
    one_hot[value] = 1.0 
    return one_hot


def prepare_dataset(filename):
    a = []
    for i in range(26):
        a.append(i)
    b = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    alphabet = {}
    for i in range(len(b)):
        alphabet[b[i]] = a[i]
    headers = ['LETTER']
    for i in range(1,17):
        headers.append(f'INPUT{i}')
    dataframe = pandas.read_csv(filename,names=headers)
    X = []
    Y = []
    Y_hot = []
    for item, row in dataframe.iterrows():
        X.append([1,row['INPUT1'],row['INPUT2'],row['INPUT3'],row['INPUT4'],row['INPUT5'],row['INPUT6'],row['INPUT7'],row['INPUT8'],
                  row['INPUT9'],row['INPUT10'],row['INPUT11'],row['INPUT12'],row['INPUT13'],row['INPUT14'],row['INPUT15'],row['INPUT16'],
                  ])
        Y.append(row['LETTER'])
    X = np.array(X)
    Y = np.array(Y)

    Y_hot = np.array([generate_one_hot(alphabet[i[0]]) for i in Y])
    return X,Y_hot
        
        

In [6]:
X,Y = prepare_dataset('data/letter-recognition.csv')
print(X.shape)
print(Y.shape)

(20000, 17)
(20000, 26)


In [15]:
model_1 = keras.Sequential([
    keras.Input(shape=(17,)),
    keras.layers.Dense(500,activation='sigmoid',name='Layer1'),
    keras.layers.Dense(26,activation='softmax', name='Layer2')
])

In [16]:
model_1.build()
model_1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer1 (Dense)              (None, 500)               9000      
                                                                 
 Layer2 (Dense)              (None, 26)                13026     
                                                                 
Total params: 22,026
Trainable params: 22,026
Non-trainable params: 0
_________________________________________________________________


In [30]:
class Linear(keras.layers.Layer):
    def __init__(self,n_outputs):
        super(Linear,self).__init__()
        self.n_outputs = n_outputs
    def build(self,input_shape):
        self.weight = self.add_weight(shape=(input_shape[-1],self.n_outputs),initializer='random_normal',trainable=True)
        self.bias = self.add_weight(shape=(self.n_outputs,),initializer='random_normal',trainable=True)

    def call(self,inputs):
        return tf.matmul(inputs,self.weight) + self.bias

In [32]:
class Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.linear_1 = Linear(500)
        self.linear_2 = Linear(26)
    def __call__(self,x,training=True):
        x = self.linear_1(x)
        output = self.linear_2(x)
        return output
    
model_class = Model()
model_class(tf.zeros([1,217])).shape

TensorShape([1, 26])

In [24]:
class Linear(keras.layers.Layer):
    def __init__(self,n_inputs,n_outputs):
        super(Linear,self).__init__()
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
    def build(self,input_shape):
        self.weight = self.add_weight(shape=(self.n_inputs,self.n_outputs),initializers='random_normal',trainable=True)
        self.bias = self.add_weight(shape=(self.n_outputs,),initializers='random_normal',trainable=True)

    def call(self,inputs):
        return tf.matmul(inputs,self.weight) + self.bias

model_class